This is a markdown for interactive code

In [ ]:
## Imports packages
import numpy as np
import pandas as pd
import os

In [2]:
# Change working directory
os.path.abspath('') # get current working directory
os.chdir('..') # change to parent directory
getdir = os.getcwd() # get current working directory after change

In [3]:
# Define input directory and initialize dictionaries
excel_files = os.listdir('Data/raw_data/ANH_gas_prod/') # create a list with all excel files names in the directory
datayear_dic = {} # create an empty dictionary to store excel per year dataframes
sheet_dict = {} # create an empty dictionary to store excel sheet dataframes
namelist = ['AÑO', 'MES', 'CAMPO', 'CONTRATO', 'EMPRESA', 'DEPARTAMENTO',
       'MUNICIPIO', 'PRODUCCION_FISCALIZADA', 'GASLIFT', 'GAS_REINYECTADO',
       'GAS_QUEMADO', 'CONSUMO_EN_CAMPO', 'ENVIADO_A_PLANTA', 'GAS_TRANSFORMADO',
       'ENTREGADO_A_GASEODUCTOS']

In [4]:
## Process each excel file and its sheets
for excel in excel_files: # loop through each excel file in the directory
    data_dir = f'Data/raw_data/ANH_gas_prod/{excel}' # create the path for each excel file
    sheet_names = pd.ExcelFile(data_dir).sheet_names[1:] # create a list with all sheet names except the first one

    for sheet in sheet_names: # loop through each sheet in the excel file
        tempdf = pd.read_excel(data_dir, sheet_name=sheet) # read each sheet into a dataframe and store it in the sheet_dict

        while True: # loop to find the header row
            if tempdf.iloc[0,0] == 'AÑO':  # check if the first cell in the first row is 'AÑO'
                tempdf.columns = tempdf.iloc[0] # set the first row as header
                tempdf = tempdf.drop(0).reset_index(drop=True) # drop the first row and reset index
                break # exit the loop
            else:
                tempdf = tempdf.drop(0).reset_index(drop=True) # drop the first row if it does not contain 'AÑO' and reset index

        tempdf = tempdf[tempdf['AÑO'] == tempdf.iloc[0,0]] # filter rows where 'AÑO' column matches the year in the first row to remove any extra footer rows

        tempdf.columns =  namelist # assign the predefined column names to the dataframe
        sheet_dict[sheet] = tempdf  # store the cleaned dataframe in the sheet_dict
        tempdf = pd.DataFrame() # reset tempdf for next iteration

    datayear_dic[excel] = pd.concat(sheet_dict.values()) # concatenate all sheets dataframes into a single dataframe per excel file and store it in the datayear_dic
    sheet_dict = {} # reset sheet_dict for next iteration

final_df = pd.concat(datayear_dic.values()) # concatenate all excel dataframes into a single final dataframe

In [5]:
# Save the final dataframe to a csv file
final_df.to_csv('Data/processed_data/ANH_gas_concat.csv', index=False) # save the final dataframe to an csv file
#final_df.to_excel('Data/processed_data/ANH_gas_concat.xlsx', index=False) # save the final dataframe to an excel file